In [1]:
# imports

from catalog import Catalog
from course import Course

from bs4 import BeautifulSoup
import requests

print('Imports Completed')

Imports Completed


In [2]:
# variable setup
catalog = Catalog()

all_courses_url = 'http://catalogue.uci.edu/allcourses/'
save_file_path = 'catalog.json'

print('Variables Initialized')

Variables Initialized


In [3]:
all_courses_page = requests.get(all_courses_url)
all_courses_soup = BeautifulSoup(all_courses_page.content, 'html.parser')

print('Soup Made')

Soup Made


In [4]:
a_to_z_index = all_courses_soup.find('div', id='atozindex')
for section in a_to_z_index.find_all('a', href=True):
    section_url = all_courses_url + section['href'].split('/')[2]
    print(section_url)
    section_page = requests.get(section_url)
    section_soup = BeautifulSoup(section_page.content, 'html.parser')

    courses = section_soup.find_all('div', class_='courseblock')

    for course in courses:
        course_block_title = course.find('p', class_='courseblocktitle')
        course_block_desc = course.find('div', class_='courseblockdesc')

        course_block_title_text = course_block_title.text.split(sep='.')
        section_code = course_block_title_text[0].split(sep='\xa0')
        section = ' '.join(section_code[:-1])
        code = section_code[-1]
        title = course_block_title_text[1].strip()
        try:
            units = course_block_title_text[2].split()[0]
        except IndexError:
            units = 0

        course_block_desc_children = course_block_desc.findChildren()
        description = course_block_desc_children[0].text

        design_units = 0
        prerequisites = []
        corequisites = []

        for block in course_block_desc_children:
            text = block.text
            if 'Prerequisite:' in text:  # Prerequisite
                if 'Corequisite:' in text:  # Prerequisite and Corequisite
                    text = text.split(sep='\n')
                    corequisites = text[0].partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(
                        sep=' and ')
                    prerequisites = text[1].partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(
                        sep=' and ')
                else:  # just Prerequisite
                    prerequisites = text.partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(sep=' and ')
            elif 'Corequisite:' in text:  # just Corequisite
                corequisites = text.partition(' ')[2].strip().replace('\xa0', ' ').split(sep='.')[0].split(sep=' and ')

        # map(lambda prerequisite: prerequisite.strip('()').split(' or '), prerequisites)
        # map(lambda corequisite:corequisite.strip('()').split(' or '), corequisites)

        catalog.add(Course(section, code, title, units, description,
                           corequisites, prerequisites))

print('Catalog Filled')

http://catalogue.uci.edu/allcourses/ac_eng
http://catalogue.uci.edu/allcourses/afam
http://catalogue.uci.edu/allcourses/anatomy
http://catalogue.uci.edu/allcourses/anthro
http://catalogue.uci.edu/allcourses/arabic
http://catalogue.uci.edu/allcourses/armn
http://catalogue.uci.edu/allcourses/art
http://catalogue.uci.edu/allcourses/art_his
http://catalogue.uci.edu/allcourses/arts
http://catalogue.uci.edu/allcourses/asianam
http://catalogue.uci.edu/allcourses/biochem
http://catalogue.uci.edu/allcourses/bio_sci
http://catalogue.uci.edu/allcourses/bats
http://catalogue.uci.edu/allcourses/bme
http://catalogue.uci.edu/allcourses/bsemd
http://catalogue.uci.edu/allcourses/bana
http://catalogue.uci.edu/allcourses/cbe
http://catalogue.uci.edu/allcourses/cbems
http://catalogue.uci.edu/allcourses/chem
http://catalogue.uci.edu/allcourses/chc_lat
http://catalogue.uci.edu/allcourses/chinese
http://catalogue.uci.edu/allcourses/engrcee
http://catalogue.uci.edu/allcourses/classic
http://catalogue.uci.edu/

In [5]:
catalog.save_to_file(save_file_path)

print('File Saved')

File Saved
